In [ ]:
import pandas as pd
import numpy as np
import addfips
import plotly.express as px
from urllib.request import urlopen
import json
import math

In [ ]:
ev_df = pd.read_csv('New_ZEV_Sales.csv')

In [ ]:
out_of_state = ev_df[ (ev_df['County'] == 'Out Of State') ].index
ev_df.drop(out_of_state , inplace=True)

In [ ]:
ev_year_df = ev_df.groupby(['Data Year','County']).sum(numeric_only = True)
ev_year_df.reset_index(inplace=True)
ev_year_df.rename(columns = {'Number of Vehicles': 'Total', 'Data Year': 'Year'}, inplace = True)

In [ ]:
af = addfips.AddFIPS()

def create_fips_col(county):
    fips = af.get_county_fips(county, state = 'California')
    return fips

# add 'fips' column to each df
ev_year_df['fips'] = ev_year_df['County'].apply(create_fips_col)

In [ ]:
first_year = ev_year_df['Year'].iloc[0]
last_year = ev_year_df['Year'].iloc[-1]

In [ ]:
counties_lst = ev_year_df['County'].unique()

In [ ]:
for year in range(first_year, last_year + 1):
    for county in counties_lst:
        if not ((ev_year_df['Year'] == year) & (ev_year_df['County'] ==  county)).any():
            temp_df = {'Year': [year], 
                       'County': [county], 
                       'Total': [0], 
                       'fips': [af.get_county_fips(county, state = 'California')]}
            temp_df = pd.DataFrame(temp_df)
            ev_year_df = pd.concat([ev_year_df, temp_df], ignore_index = True)

ev_year_df = ev_year_df.sort_values(by = 'Year')

In [ ]:
def cumulative_sum(row):
    cur_year = row['Year']
    county = row['County']
    cum_sum = ev_year_df[(ev_year_df['Year'] <= cur_year) & (ev_year_df['County'] == county)]['Total'].sum(numeric_only=True)
    return cum_sum

In [ ]:
ev_year_df['Cumulative Total'] = ev_year_df.apply(cumulative_sum, axis=1)

In [ ]:
def log_0(num):
    if num == 0:
        return 0
    else:
        return math.log(num)

In [ ]:
ev_year_df['log_Cum_Total'] = ev_year_df['Cumulative Total'].apply(log_0)

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
ev_year_fig = px.choropleth(
    ev_year_df,
    geojson = counties,
    locations = 'fips',
    color = 'log_Cum_Total',
    scope = 'usa',
    color_continuous_scale= 'agsunset_r',
    hover_name = ev_year_df['County'],
    hover_data= {'Cumulative Total': True,
                 'fips': False,
                 'log_Cum_Total': False
                 },
    basemap_visible=True,
    animation_frame='Year'
           
)

ev_year_fig.update_geos(fitbounds = 'locations')
ev_year_fig.update_layout(height=500,margin={"r":0,"t":5,"l":0,"b":0})
ev_year_fig.layout.coloraxis.colorbar = {'title': 'Number of EVs',
                                 'x': 0.9,
                                 'tickvals': [0, 0.6, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
                                 'ticktext': [0, 2, 3] + [str(int(round(math.exp(val), -1))) for val in range(2,14)]
                                 }
ev_year_fig.show()